**Run this to delete the entire filesystem**

In [ ]:
# import shutil

# shutil.rmtree('test/')
# shutil.rmtree('train/')

**Import**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
from shutil import copy2

%matplotlib inline
import cv2
from scipy.stats import itemfreq
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns # visualizations

import random # for setting seed

import IPython

import tensorflow
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
# from confusion_matrix import plot_confusion_matrix
# from plot_history import plot
from keras.applications.vgg16 import VGG16

from keras import metrics
import keras.backend as K
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.applications import VGG16

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Functions



In [1]:

def precision(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2
    return precision

def recall(y_true, y_pred):
    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many relevant items are selected?
    recall = c1 / c3
    return recall

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def topk(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [2]:
# # Plot training & validation accuracy values
def plot(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

# Goal: Create a model that can identify an genre given a painting from the ["Painter By Numbers" dataset](https://www.kaggle.com/c/painter-by-numbers)

# Methods / Tools
1. [Keras](https://keras.io/)
2. [Tensorflow](https://www.tensorflow.org/)
3. [ResNet50 & VGG]
  * a Convolutional Neural Network (CNN) model instance in Keras
4. [Transfer Learning](https://www.kaggle.com/dansbecker/transfer-learning)


# Set a random seed and get environment info for reproducibility

In [3]:
my_seed = 42 # 480 could work too
random.seed(my_seed)
np.random.seed(my_seed)
tensorflow.set_random_seed(my_seed)

In [4]:
# print system information (but not packages)
print(IPython.sys_info())

# get module information
!pip freeze > frozen-requirements.txt

# append system information to file
with open("frozen-requirements.txt", "a") as file:
    file.write(IPython.sys_info())

{'commit_hash': '7f10f7bb3',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/opt/conda/lib/python3.6/site-packages/IPython',
 'ipython_version': '6.4.0',
 'os_name': 'posix',
 'platform': 'Linux-4.9.0-5-amd64-x86_64-with-debian-9.4',
 'sys_executable': '/opt/conda/bin/python',
 'sys_platform': 'linux',
 'sys_version': '3.6.6 |Anaconda, Inc.| (default, Oct  9 2018, 12:34:16) \n'
                '[GCC 7.3.0]'}


# Info about ResNet50 & VGG on Keras
### arguments:
* "The default **input** size for this model is **224x224**."
* "**input_tensor**: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model."
* "**classes**: optional number of classes to classify images into, only to be specified if include_top is  True, and if no weights argument is specified."

# Read in CSV File, Filter to only get top 9 genres, split into training and test set

In [5]:
data_dir = '../input/processed-data/train_processed/'
train_dir = data_dir + 'train/'
df = pd.read_csv('../input/painter-by-numbers/train_info.csv')
print("there are " + str(df.shape[0]) + " paintings inside train_csv") 
df['genre'] = df['genre'].apply(lambda x: 'nude' if x == 'nude painting (nu)' else x)

there are 79433 paintings inside train_csv


In [11]:
#filter genre_data
genre = ['portrait', 'landscape', 'genre painting', 'abstract', 'religious painting', 'cityscape', 'sketch and study', 'illustration', 'still life']
genre_data = df[df['genre'].isin(genre)]

In [12]:
genre_train_data, genre_test_data = train_test_split(genre_data, test_size=0.2)

# Create working directories for genres 

In [ ]:
working_train_dir = "train/"
working_test_dir = "test/"
if (os.path.isdir(working_train_dir) == False):
    os.mkdir(working_train_dir)
    print("created " + working_train_dir)
else:
    print(working_train_dir + " exists")
    
if (os.path.isdir(working_test_dir) == False):
    os.mkdir(working_test_dir)
    print("created " + working_test_dir)
else:
    print(working_test_dir + " exists")

for genre in genre:
    train_genre_dir = working_train_dir + genre
    if (os.path.isdir(train_genre_dir) == False):
        os.mkdir(train_genre_dir)
        print("created " + train_genre_dir)
    else:
        print(train_genre_dir + " exists")
        
    test_genre_dir = working_test_dir + genre
    if (os.path.isdir(test_genre_dir) == False):
        os.mkdir(test_genre_dir)
        print("created " + test_genre_dir)
    else:
        print(test_genre_dir + " exists")

# Copy pre-processed photos into training and test direcs 

In [ ]:
i=0
for _, row in genre_train_data.iterrows():
    f = row['filename']
    g = row['genre']
    if (os.path.exists(train_dir+f) and not os.path.exists(working_train_dir+g+'/'+f)):
        # print()
        copy2(train_dir+f, working_train_dir+g)
        i += 1

print("\ncopied train__data " + str(i))

i=0
for _, row in genre_test_data.iterrows():
    f = row['filename']
    g = row['genre']
    if (os.path.exists(train_dir+f) and not os.path.exists(working_test_dir+g+'/'+f)):
        copy2(train_dir+f, working_test_dir+g)
        i += 1

print("\ncopied test_data " + str(i))


steps_per_epoch = 752
validation_steps = 188

# Create data generators

In [ ]:
image_size_224 = 224

data_generator_224_no_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator_224_no_aug = data_generator_224_no_aug.flow_from_directory(
        working_train_dir,
        target_size=(image_size_224, image_size_224),
        batch_size=64,
        class_mode='categorical', 
        shuffle=False)

validation_generator_224_no_aug = data_generator_224_no_aug.flow_from_directory(
        working_test_dir,
        target_size=(image_size_224, image_size_224),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

print("\n\nmodel - train_generator_no_aug")

In [4]:
data_generator_224_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True)

train_generator_224_aug = data_generator_224_aug.flow_from_directory(
        working_train_dir,
        target_size=(image_size_224, image_size_224),
        batch_size=64,
        class_mode='categorical', 
        shuffle=False)

validation_generator_224_aug = data_generator_224_aug.flow_from_directory(
        working_test_dir,
        target_size=(image_size_224, image_size_224),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

# Simple Model

In [ ]:
num_classes = len(os.listdir(working_test_dir)) 

In [ ]:
def basic_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # the model so far outputs 3D feature maps (height, width, features)

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(9))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-3, decay=1e-3, momentum=0.9),
              metrics=['accuracy'])
    
    return model
cnn_model = basic_cnn_model()

In [ ]:
simple_history = cnn_model.fit_generator(
        train_generator_224_aug,
        steps_per_epoch=steps_per_epoch,
        epochs=50,
        validation_data=validation_generator_224_aug,
        validation_steps= validation_steps
)

In [ ]:
plot(history)

# VGG

In [ ]:
weights_notop_path_vgg = '../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

## Non-Augmented

In [ ]:
model_vgg_no_aug = Sequential()

# now let's set up the first layers
model_vgg_no_aug.add(VGG16(    # add a whole ResNet50 model
  include_top=False,          # without the last layer
  weights=weights_notop_path_vgg, # and with the "notop" weights file
  pooling='avg' # means collapse extra "channels" into 1D tensor by taking an avg across channels
))

# Now lets add a "Dense" layer to make predictions
model_vgg_no_aug.add(Dense(
  num_classes, # this last layer just has 2 nodes
  activation='softmax' # apply softmax function to turn values of this layer into probabilities
))

model_vgg_no_aug.layers[0].trainable = False

model_vgg_no_aug.compile(
  optimizer='sgd', # stochastic gradient descent (how to update Dense connections during training)
  loss='categorical_crossentropy', # aka "log loss" -- the cost function to minimize 
  # so 'optimizer' algorithm will minimize 'loss' function
  metrics=['accuracy', f1_score, topk, recall, precision] # ask it to report % of correct predictions
)
print("Compiled layer")

In [ ]:
history_vgg_no_aug = model_vgg_no_aug.fit_generator(
        train_generator_224_no_aug,
        steps_per_epoch=steps_per_epoch,
        epochs=50, # so... total of 9 epochs?
        validation_data=validation_generator_224_no_aug,
        validation_steps=validation_steps)

In [ ]:
plot(history_vgg_no_aug)

## Augmented

In [ ]:
model_vgg_aug = Sequential()

# now let's set up the first layers
model_vgg_aug.add(VGG16(    # add a whole ResNet50 model
  include_top=False,          # without the last layer
  weights=weights_notop_path_vgg, # and with the "notop" weights file
  pooling='avg' # means collapse extra "channels" into 1D tensor by taking an avg across channels
))

# Now lets add a "Dense" layer to make predictions
model_vgg_aug.add(Dense(
  num_classes, # this last layer just has 2 nodes
  activation='softmax' # apply softmax function to turn values of this layer into probabilities
))

model_vgg_aug.layers[0].trainable = False

model_vgg_aug.compile(
  optimizer='sgd', # stochastic gradient descent (how to update Dense connections during training)
  loss='categorical_crossentropy', # aka "log loss" -- the cost function to minimize 
  # so 'optimizer' algorithm will minimize 'loss' function
  metrics=['accuracy', f1_score, topk, recall, precision] # ask it to report % of correct predictions
)
print("Compiled layer")

In [ ]:
history_vgg_aug = model_vgg_no_aug.fit_generator(
        train_generator_224_aug,
        steps_per_epoch=steps_per_epoch,
        epochs=50, # so... total of 9 epochs?
        validation_data=validation_generator_224_aug,
        validation_steps=validation_steps)

In [ ]:
plot(history_vgg_aug)

# ResNet50

## Specify Model

In [ ]:
weights_notop_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

### Non Augmented

In [ ]:
model_rs_no_aug = Sequential()

# now let's set up the first layers
model_rs_no_aug.add(ResNet50(    # add a whole ResNet50 model
  include_top=False,          # without the last layer
  weights=weights_notop_path, # and with the "notop" weights file
  pooling='avg' # means collapse extra "channels" into 1D tensor by taking an avg across channels
))

# Now lets add a "Dense" layer to make predictions
model_rs_no_aug.add(Dense(
  num_classes, # this last layer just has 2 nodes
  activation='softmax' # apply softmax function to turn values of this layer into probabilities
))

model_rs_no_aug.layers[0].trainable = False

model_rs_no_aug.compile(
  optimizer='sgd', # stochastic gradient descent (how to update Dense connections during training)
  loss='categorical_crossentropy', # aka "log loss" -- the cost function to minimize 
  # so 'optimizer' algorithm will minimize 'loss' function
  metrics=['accuracy', f1_score, topk, recall, precision] # ask it to report % of correct predictions
)
print("Compiled layer")

In [ ]:
history_resnet_no_aug = model_rs_no_aug.fit_generator(
        train_generator_224_no_aug,
        steps_per_epoch=200,
        epochs=50, 
        validation_data=validation_generator_224_no_aug,
        validation_steps=80)
print("history")

In [ ]:
plot(history_resnet_no_aug)

### Augmented

In [ ]:
model_rs_aug = Sequential()

# now let's set up the first layers
model_rs_aug.add(ResNet50(    # add a whole ResNet50 model
  include_top=False,          # without the last layer
  weights=weights_notop_path, # and with the "notop" weights file
  pooling='avg' # means collapse extra "channels" into 1D tensor by taking an avg across channels
))

# Now lets add a "Dense" layer to make predictions
model_rs_aug.add(Dense(
  num_classes, # this last layer just has 2 nodes
  activation='softmax' # apply softmax function to turn values of this layer into probabilities
))

model_rs_aug.layers[0].trainable = False

model_rs_aug.compile(
  optimizer='sgd', # stochastic gradient descent (how to update Dense connections during training)
  loss='categorical_crossentropy', # aka "log loss" -- the cost function to minimize 
  # so 'optimizer' algorithm will minimize 'loss' function
  metrics=['accuracy', f1_score, topk, recall, precision] # ask it to report % of correct predictions
)
print("Compiled layer")

In [ ]:
history_resnet_aug = model_rs_aug.fit_generator(
        train_generator_224_aug,
        steps_per_epoch=200,
        epochs=50, 
        validation_data=validation_generator_224_aug,
        validation_steps=80)
print("history")

In [ ]:
plot(history_resnet_aug)

# Confusion Matrix

In [ ]:
Y_pred = model_rs_no_aug.predict_generator(validation_generator_224_no_aug)
Y_trues = validation_generator_224_no_aug.classes

In [ ]:
cm = confusion_matrix(Y_pred, Y_trues)

plot_confusion_matrix(cm, genre)